# MLOps Zoomcamp: Intro
### Homework

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

In [2]:
df_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [3]:
print(df_jan.shape[1])

19


#### Q2. Computing duration

In [4]:
def compute_duration(df):
    df = df.copy()
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda x: x.total_seconds() / 60)

    return df

In [5]:
df_jan = compute_duration(df_jan)
df_feb = compute_duration(df_feb)

In [6]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [7]:
print(df_jan.duration.std().round(2))

42.59


#### Q3. Dropping outliers

In [8]:
def drop_outliers(df):
    df = df.loc[(df.duration >= 1) & (df.duration <= 60), :]
    return df

In [9]:
print(df_jan.shape[0])

3066766


In [10]:
df_jan = drop_outliers(df_jan)
df_feb = drop_outliers(df_feb)

In [11]:
print(df_jan.shape[0])

3009173


In [12]:
round(3009173 / 3066766, 2)

0.98

In [13]:
def one_hot_encoding(df, encoder=None):
    df_categorical = df.select_dtypes(exclude='number')
    df_numerical = df.select_dtypes(include='number')

    if not encoder:
        encoder = OneHotEncoder(dtype=int, handle_unknown='infrequent_if_exist')
        encoder.fit(df_categorical)

    encodings = encoder.transform(df_categorical).toarray()
    df_encodings = pd.DataFrame(data=encodings, columns=encoder.get_feature_names_out())

    df_encoded = pd.concat([df_numerical, df_encodings], axis=1).dropna()

    return df_encoded, encoder

In [14]:
df_jan = df_jan[['PULocationID', 'DOLocationID', 'duration']]
df_feb = df_feb[['PULocationID', 'DOLocationID', 'duration']]

In [15]:
df_jan[['PULocationID', 'DOLocationID']] = df_jan[['PULocationID', 'DOLocationID']].astype("category")
df_feb[['PULocationID', 'DOLocationID']] = df_feb[['PULocationID', 'DOLocationID']].astype("category")

In [16]:
df_jan_encoded, encoder = one_hot_encoding(df_jan)
df_feb_encoded, _ = one_hot_encoding(df_feb, encoder)

#### Q5. Training a model

In [17]:
X_train = df_jan_encoded.drop('duration', axis=1)
y_train = df_jan_encoded.duration

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_train)
root_mean_squared_error(y_train, y_pred)

#### Q6. Evaluating the model

In [ ]:
X_test = df_feb_encoded.drop('duration', axis=1)
y_test = df_feb_encoded.duration

In [ ]:
y_pred = model.predict(X_test)
root_mean_squared_error(y_test, y_pred)